# Parameters and Preparing the Data

In [ ]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import (
    layers,
    models,
    datasets,
    callbacks,
    losses,
    optimizers,
    metrics,
)
from scipy.stats import norm
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

# Load data
train_data = loadmat('train_32x32.mat')
test_data = loadmat('test_32x32.mat')

# Preprocess the data
x_train = np.transpose(train_data['X'], (3, 0, 1, 2)).astype('float32') / 255.0
y_train = train_data['y'].flatten()
x_test = np.transpose(test_data['X'], (3, 0, 1, 2)).astype('float32') / 255.0
y_test = test_data['y'].flatten()

# Display the shape of the datasets
print(f'Training data shape: {x_train.shape}')
print(f'Test data shape: {x_test.shape}')

IMAGE_SIZE = 32
BATCH_SIZE = 100
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 2
EPOCHS = 50
BETA = 500

--2024-10-11 22:07:10--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  8.23MB/s    in 21s     

2024-10-11 22:07:31 (8.26 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2024-10-11 22:07:31--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  5.23MB/s    in 14s     

2024-10-11 22:07:46 (4.36 MB/s) - ‘test_32x32.mat’ saved [64275384/64

# Variational Autoencoder

In [ ]:
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Encoder

In [ ]:
# Encoder
encoder_input = layers.Input(
    shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="encoder_input"
)
x = layers.Conv2D(32, (3, 3), strides=2, activation="relu", padding="same")(
    encoder_input
)
x = layers.Conv2D(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(128, (3, 3), strides=2, activation="relu", padding="same")(x)
shape_before_flattening = K.int_shape(x)[1:]  # the decoder will need this!

x = layers.Flatten()(x)
z_mean = layers.Dense(EMBEDDING_DIM, name="z_mean")(x)
z_log_var = layers.Dense(EMBEDDING_DIM, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_input             │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 16, 16, 32)     │            896 │ encoder_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 8, 8, 64)       │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 4, 4, 128)      │         73,856 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 2048)           │              0 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 2)              │          4,098 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 2)              │          4,098 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 2)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 101,444 (396.27 KB)

 Trainable params: 101,444 (396.27 KB)

 Non-trainable params: 0 (0.00 B)

# Decoder

In [ ]:
# Decoder

decoder_input = layers.Input(shape=(EMBEDDING_DIM,), name="decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(
    128, (3, 3), strides=2, activation="relu", padding="same"
)(x)
x = layers.Conv2DTranspose(
    64, (3, 3), strides=2, activation="relu", padding="same"
)(x)
x = layers.Conv2DTranspose(
    32, (3, 3), strides=2, activation="relu", padding="same"
)(x)
decoder_output = layers.Conv2D(
    3,
    (3, 3),
    strides=1,
    activation="sigmoid",
    padding="same",
    name="decoder_output",
)(x)

decoder = models.Model(decoder_input, decoder_output)
decoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ decoder_input (InputLayer)           │ (None, 2)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2048)                │           6,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 16, 16, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 32, 32, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_output (Conv2D)              │ (None, 32, 32, 3)           │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 246,851 (964.26 KB)

 Trainable params: 246,851 (964.26 KB)

 Non-trainable params: 0 (0.00 B)

# Build VAE

In [ ]:
class build_vae(models.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(build_vae, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def call(self, inputs):
        """Call the model on a particular input."""
        z_mean, z_log_var, z = encoder(inputs)
        reconstruction = decoder(z)
        return z_mean, z_log_var, reconstruction

    def train_step(self, data):
        """Step run during training."""
        with tf.GradientTape() as tape:
            z_mean, z_log_var, reconstruction = self(data)
            reconstruction_loss = tf.reduce_mean(
                BETA
                * losses.binary_crossentropy(
                    data, reconstruction, axis=(1, 2, 3)
                )
            )
            kl_loss = tf.reduce_mean(
                tf.reduce_sum(
                    -0.5
                    * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
                    axis=1,
                )
            )
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        """Step run during validation."""
        if isinstance(data, tuple):
            data = data[0]

        z_mean, z_log_var, reconstruction = self(data)
        reconstruction_loss = tf.reduce_mean(
            BETA
            * losses.binary_crossentropy(data, reconstruction, axis=(1, 2, 3))
        )
        kl_loss = tf.reduce_mean(
            tf.reduce_sum(
                -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
                axis=1,
            )
        )
        total_loss = reconstruction_loss + kl_loss

        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

# Create a variational autoencoder
vae = build_vae(encoder, decoder)

# Training

In [ ]:
# Compile the variational autoencoder
optimizer = optimizers.Adam(learning_rate=0.0005)
vae.compile(optimizer=optimizer)

latent_dims = [2. 5, 10]
for dim in latent_dims:
    print(f'Training VAE with latent dimension: {dim}')
    vae = build_vae(encoder, decoder)
    vae.compile(optimizer='adam', loss=losses.binary_crossentropy)
    vae.fit(x_train, epochs=50, batch_size=128, validation_data=(x_test, x_test))

# Reconstruct

In [ ]:
# Select a subset of the test set
n_to_predict = 5000
example_images = x_test[:n_to_predict]
example_labels = y_test[:n_to_predict]

# Create autoencoder predictions and display
z_mean, z_log_var, reconstructions = vae.predict(example_images)
print("Example real clothing items")
display(example_images)
print("Reconstructions")
display(reconstructions)

# Embed with Encoder

In [ ]:
# Encode the example images
z_mean, z_var, z = encoder.predict(example_images)

# Show the encoded points in 2D space
figsize = 8

plt.figure(figsize=(figsize, figsize))
plt.scatter(z[:, 0], z[:, 1], c="black", alpha=0.5, s=3)
plt.show()

# Generate with Decoder

In [ ]:
# Sample some points in the latent space, from the standard normal distribution
grid_width, grid_height = (6, 3)
z_sample = np.random.normal(size=(grid_width * grid_height, 2))

# Decode the sampled points
reconstructions = decoder.predict(z_sample)

# Convert original embeddings and sampled embeddings to p-values
p = norm.cdf(z)
p_sample = norm.cdf(z_sample)

# Draw a plot of...
figsize = 8
plt.figure(figsize=(figsize, figsize))

# ... the original embeddings ...
plt.scatter(z[:, 0], z[:, 1], c="black", alpha=0.5, s=2)

# ... and the newly generated points in the latent space
plt.scatter(z_sample[:, 0], z_sample[:, 1], c="#00B0F0", alpha=1, s=40)
plt.show()

# Add underneath a grid of the decoded images
fig = plt.figure(figsize=(figsize, grid_height * 2))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i in range(grid_width * grid_height):
    ax = fig.add_subplot(grid_height, grid_width, i + 1)
    ax.axis("off")
    ax.text(
        0.5,
        -0.35,
        str(np.round(z_sample[i, :], 1)),
        fontsize=10,
        ha="center",
        transform=ax.transAxes,
    )
    ax.imshow(reconstructions[i, :, :], cmap="Greys")

# Discusion

5 epochs:
Training VAE with latent dimension: 2
Epoch 1/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 188s 324ms/step - kl_loss: 1.4889 - reconstruction_loss: 332.0292 - total_loss: 333.5181 - val_kl_loss: 2.2917 - val_loss: 323.7550 - val_reconstruction_loss: 321.4633
Epoch 2/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 185s 323ms/step - kl_loss: 2.5196 - reconstruction_loss: 321.5546 - total_loss: 324.0742 - val_kl_loss: 2.4504 - val_loss: 320.0995 - val_reconstruction_loss: 317.6491
Epoch 3/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 204s 326ms/step - kl_loss: 2.6471 - reconstruction_loss: 319.9093 - total_loss: 322.5563 - val_kl_loss: 2.9072 - val_loss: 319.2141 - val_reconstruction_loss: 316.3069
Epoch 4/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 186s 325ms/step - kl_loss: 2.7792 - reconstruction_loss: 318.5204 - total_loss: 321.2996 - val_kl_loss: 2.8323 - val_loss: 316.2164 - val_reconstruction_loss: 313.3841
Epoch 5/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 201s 324ms/step - kl_loss: 2.8115 - reconstruction_loss: 318.0206 - total_loss: 320.8321 - val_kl_loss: 2.8889 - val_loss: 316.7099 - val_reconstruction_loss: 313.8210
Training VAE with latent dimension: 5
Epoch 1/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 197s 339ms/step - kl_loss: 2.8055 - reconstruction_loss: 318.6307 - total_loss: 321.4363 - val_kl_loss: 3.0424 - val_loss: 316.0744 - val_reconstruction_loss: 313.0321
Epoch 2/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 197s 331ms/step - kl_loss: 2.8432 - reconstruction_loss: 318.1993 - total_loss: 321.0425 - val_kl_loss: 3.0007 - val_loss: 316.2474 - val_reconstruction_loss: 313.2467
Epoch 3/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 187s 326ms/step - kl_loss: 2.8262 - reconstruction_loss: 318.0373 - total_loss: 320.8636 - val_kl_loss: 2.8875 - val_loss: 316.4171 - val_reconstruction_loss: 313.5295
Epoch 4/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 190s 331ms/step - kl_loss: 2.8308 - reconstruction_loss: 317.9869 - total_loss: 320.8176 - val_kl_loss: 2.9980 - val_loss: 316.1076 - val_reconstruction_loss: 313.1096
Epoch 5/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 202s 332ms/step - kl_loss: 2.8364 - reconstruction_loss: 318.0974 - total_loss: 320.9339 - val_kl_loss: 2.9793 - val_loss: 315.8542 - val_reconstruction_loss: 312.8749
Training VAE with latent dimension: 10
Epoch 1/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 191s 329ms/step - kl_loss: 2.8385 - reconstruction_loss: 318.4011 - total_loss: 321.2397 - val_kl_loss: 3.0503 - val_loss: 316.5591 - val_reconstruction_loss: 313.5088
Epoch 2/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 203s 331ms/step - kl_loss: 2.8732 - reconstruction_loss: 317.7626 - total_loss: 320.6358 - val_kl_loss: 2.8578 - val_loss: 316.1581 - val_reconstruction_loss: 313.3003
Epoch 3/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 200s 327ms/step - kl_loss: 2.8642 - reconstruction_loss: 318.0220 - total_loss: 320.8862 - val_kl_loss: 3.0882 - val_loss: 316.3940 - val_reconstruction_loss: 313.3058
Epoch 4/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 223s 364ms/step - kl_loss: 2.8852 - reconstruction_loss: 318.0053 - total_loss: 320.8907 - val_kl_loss: 3.0334 - val_loss: 316.1222 - val_reconstruction_loss: 313.0888
Epoch 5/5
573/573 ━━━━━━━━━━━━━━━━━━━━ 243s 331ms/step - kl_loss: 2.8960 - reconstruction_loss: 318.0488 - total_loss: 320.9447 - val_kl_loss: 3.0478 - val_loss: 316.5784 - val_reconstruction_loss: 313.5305

20 epochs:
raining VAE with latent dimension: 2
Epoch 1/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 376s 647ms/step - kl_loss: 1.6232 - reconstruction_loss: 330.5131 - total_loss: 332.1364 - val_kl_loss: 2.9420 - val_loss: 316.2372 - val_reconstruction_loss: 313.2953
Epoch 2/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 326s 568ms/step - kl_loss: 2.7798 - reconstruction_loss: 318.5995 - total_loss: 321.3793 - val_kl_loss: 2.8409 - val_loss: 315.8895 - val_reconstruction_loss: 313.0487
Epoch 3/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 384s 572ms/step - kl_loss: 2.7976 - reconstruction_loss: 318.3217 - total_loss: 321.1194 - val_kl_loss: 2.9517 - val_loss: 316.2375 - val_reconstruction_loss: 313.2858
Epoch 4/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 382s 573ms/step - kl_loss: 2.8213 - reconstruction_loss: 318.0779 - total_loss: 320.8992 - val_kl_loss: 3.0000 - val_loss: 316.2977 - val_reconstruction_loss: 313.2977
Epoch 5/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 384s 577ms/step - kl_loss: 2.8144 - reconstruction_loss: 318.1392 - total_loss: 320.9536 - val_kl_loss: 2.7953 - val_loss: 316.1249 - val_reconstruction_loss: 313.3297
Epoch 6/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 383s 579ms/step - kl_loss: 2.8418 - reconstruction_loss: 317.9790 - total_loss: 320.8208 - val_kl_loss: 2.9035 - val_loss: 316.1716 - val_reconstruction_loss: 313.2681
Epoch 7/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 380s 576ms/step - kl_loss: 2.8357 - reconstruction_loss: 318.0905 - total_loss: 320.9261 - val_kl_loss: 2.9580 - val_loss: 315.9011 - val_reconstruction_loss: 312.9431
Epoch 8/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 387s 584ms/step - kl_loss: 2.8442 - reconstruction_loss: 318.1350 - total_loss: 320.9791 - val_kl_loss: 3.0948 - val_loss: 316.4609 - val_reconstruction_loss: 313.3661
Epoch 9/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 377s 574ms/step - kl_loss: 2.8532 - reconstruction_loss: 318.0838 - total_loss: 320.9369 - val_kl_loss: 2.8894 - val_loss: 316.6727 - val_reconstruction_loss: 313.7833
Epoch 10/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 384s 578ms/step - kl_loss: 2.8650 - reconstruction_loss: 318.1479 - total_loss: 321.0130 - val_kl_loss: 2.7826 - val_loss: 316.1878 - val_reconstruction_loss: 313.4053
Epoch 11/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 375s 566ms/step - kl_loss: 2.8881 - reconstruction_loss: 317.8177 - total_loss: 320.7058 - val_kl_loss: 3.0117 - val_loss: 316.1812 - val_reconstruction_loss: 313.1695
Epoch 12/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 386s 572ms/step - kl_loss: 2.8890 - reconstruction_loss: 317.7844 - total_loss: 320.6734 - val_kl_loss: 3.0110 - val_loss: 315.7556 - val_reconstruction_loss: 312.7446
Epoch 13/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 382s 573ms/step - kl_loss: 2.9060 - reconstruction_loss: 317.7208 - total_loss: 320.6267 - val_kl_loss: 3.2684 - val_loss: 316.4734 - val_reconstruction_loss: 313.2050
Epoch 14/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 383s 575ms/step - kl_loss: 2.9052 - reconstruction_loss: 318.1915 - total_loss: 321.0967 - val_kl_loss: 3.0651 - val_loss: 315.9651 - val_reconstruction_loss: 312.9000
Epoch 15/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 372s 558ms/step - kl_loss: 2.9211 - reconstruction_loss: 317.8592 - total_loss: 320.7803 - val_kl_loss: 3.0775 - val_loss: 315.8050 - val_reconstruction_loss: 312.7275
Epoch 16/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 319s 558ms/step - kl_loss: 2.9208 - reconstruction_loss: 317.8479 - total_loss: 320.7686 - val_kl_loss: 3.1294 - val_loss: 316.1984 - val_reconstruction_loss: 313.0690
Epoch 17/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 330s 576ms/step - kl_loss: 2.9351 - reconstruction_loss: 317.7010 - total_loss: 320.6361 - val_kl_loss: 2.8882 - val_loss: 316.3633 - val_reconstruction_loss: 313.4751
Epoch 18/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 382s 576ms/step - kl_loss: 2.9437 - reconstruction_loss: 317.6518 - total_loss: 320.5955 - val_kl_loss: 3.0688 - val_loss: 316.1057 - val_reconstruction_loss: 313.0368
Epoch 19/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 383s 578ms/step - kl_loss: 2.9506 - reconstruction_loss: 317.6852 - total_loss: 320.6358 - val_kl_loss: 3.0309 - val_loss: 316.2992 - val_reconstruction_loss: 313.2683
Epoch 20/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 380s 575ms/step - kl_loss: 2.9517 - reconstruction_loss: 317.7843 - total_loss: 320.7361 - val_kl_loss: 3.1051 - val_loss: 316.1707 - val_reconstruction_loss: 313.0656
Training VAE with latent dimension: 5
Epoch 1/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 333s 573ms/step - kl_loss: 2.9427 - reconstruction_loss: 317.9989 - total_loss: 320.9417 - val_kl_loss: 3.1247 - val_loss: 316.0618 - val_reconstruction_loss: 312.9372
Epoch 2/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 387s 583ms/step - kl_loss: 2.9630 - reconstruction_loss: 317.6205 - total_loss: 320.5834 - val_kl_loss: 2.9874 - val_loss: 316.4770 - val_reconstruction_loss: 313.4896
Epoch 3/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 325s 567ms/step - kl_loss: 2.9650 - reconstruction_loss: 317.7124 - total_loss: 320.6773 - val_kl_loss: 3.0012 - val_loss: 316.3191 - val_reconstruction_loss: 313.3179
Epoch 4/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 377s 559ms/step - kl_loss: 2.9801 - reconstruction_loss: 317.3668 - total_loss: 320.3469 - val_kl_loss: 3.2574 - val_loss: 316.1561 - val_reconstruction_loss: 312.8988
Epoch 5/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 324s 562ms/step - kl_loss: 2.9838 - reconstruction_loss: 317.7395 - total_loss: 320.7232 - val_kl_loss: 3.0498 - val_loss: 316.4323 - val_reconstruction_loss: 313.3825
Epoch 6/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 388s 573ms/step - kl_loss: 2.9784 - reconstruction_loss: 317.5721 - total_loss: 320.5504 - val_kl_loss: 3.0290 - val_loss: 315.7408 - val_reconstruction_loss: 312.7118
Epoch 7/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 385s 578ms/step - kl_loss: 2.9864 - reconstruction_loss: 317.5363 - total_loss: 320.5227 - val_kl_loss: 3.1390 - val_loss: 316.1323 - val_reconstruction_loss: 312.9933
Epoch 8/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 369s 555ms/step - kl_loss: 3.0127 - reconstruction_loss: 317.6545 - total_loss: 320.6673 - val_kl_loss: 3.1303 - val_loss: 316.0165 - val_reconstruction_loss: 312.8862
Epoch 9/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 327s 571ms/step - kl_loss: 2.9917 - reconstruction_loss: 317.5669 - total_loss: 320.5587 - val_kl_loss: 3.1106 - val_loss: 315.6818 - val_reconstruction_loss: 312.5712
Epoch 10/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 389s 584ms/step - kl_loss: 3.0148 - reconstruction_loss: 317.5702 - total_loss: 320.5849 - val_kl_loss: 3.0235 - val_loss: 316.2617 - val_reconstruction_loss: 313.2382
Epoch 11/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 381s 582ms/step - kl_loss: 2.9943 - reconstruction_loss: 317.5787 - total_loss: 320.5729 - val_kl_loss: 3.1261 - val_loss: 315.9146 - val_reconstruction_loss: 312.7885
Epoch 12/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 382s 583ms/step - kl_loss: 3.0247 - reconstruction_loss: 317.5397 - total_loss: 320.5644 - val_kl_loss: 3.1479 - val_loss: 316.1145 - val_reconstruction_loss: 312.9666
Epoch 13/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 378s 577ms/step - kl_loss: 3.0258 - reconstruction_loss: 317.6719 - total_loss: 320.6977 - val_kl_loss: 3.1367 - val_loss: 315.9900 - val_reconstruction_loss: 312.8533
Epoch 14/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 380s 574ms/step - kl_loss: 3.0278 - reconstruction_loss: 317.4988 - total_loss: 320.5266 - val_kl_loss: 3.1211 - val_loss: 316.1022 - val_reconstruction_loss: 312.9811
Epoch 15/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 383s 577ms/step - kl_loss: 3.0171 - reconstruction_loss: 317.5217 - total_loss: 320.5387 - val_kl_loss: 3.0186 - val_loss: 316.1509 - val_reconstruction_loss: 313.1324
Epoch 16/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 381s 575ms/step - kl_loss: 3.0380 - reconstruction_loss: 317.4799 - total_loss: 320.5178 - val_kl_loss: 3.1603 - val_loss: 315.9740 - val_reconstruction_loss: 312.8138
Epoch 17/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 381s 574ms/step - kl_loss: 3.0359 - reconstruction_loss: 317.7234 - total_loss: 320.7594 - val_kl_loss: 3.1470 - val_loss: 315.9899 - val_reconstruction_loss: 312.8429
Epoch 18/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 328s 572ms/step - kl_loss: 3.0536 - reconstruction_loss: 317.3428 - total_loss: 320.3965 - val_kl_loss: 3.2743 - val_loss: 315.9290 - val_reconstruction_loss: 312.6547
Epoch 19/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 390s 586ms/step - kl_loss: 3.0523 - reconstruction_loss: 317.4300 - total_loss: 320.4821 - val_kl_loss: 3.2177 - val_loss: 316.3752 - val_reconstruction_loss: 313.1576
Epoch 20/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 376s 575ms/step - kl_loss: 3.0481 - reconstruction_loss: 317.6314 - total_loss: 320.6796 - val_kl_loss: 3.1179 - val_loss: 316.1641 - val_reconstruction_loss: 313.0462
Training VAE with latent dimension: 10
Epoch 1/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 354s 611ms/step - kl_loss: 3.0463 - reconstruction_loss: 317.7028 - total_loss: 320.7491 - val_kl_loss: 3.0518 - val_loss: 315.6212 - val_reconstruction_loss: 312.5694
Epoch 2/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 364s 581ms/step - kl_loss: 3.0641 - reconstruction_loss: 317.4104 - total_loss: 320.4745 - val_kl_loss: 3.1827 - val_loss: 315.7643 - val_reconstruction_loss: 312.5816
Epoch 3/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 383s 583ms/step - kl_loss: 3.0538 - reconstruction_loss: 317.3911 - total_loss: 320.4448 - val_kl_loss: 3.2797 - val_loss: 316.0664 - val_reconstruction_loss: 312.7867
Epoch 4/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 379s 577ms/step - kl_loss: 3.0692 - reconstruction_loss: 317.6298 - total_loss: 320.6989 - val_kl_loss: 3.1866 - val_loss: 315.9988 - val_reconstruction_loss: 312.8122
Epoch 5/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 381s 576ms/step - kl_loss: 3.0629 - reconstruction_loss: 317.5539 - total_loss: 320.6169 - val_kl_loss: 3.1510 - val_loss: 315.8744 - val_reconstruction_loss: 312.7234
Epoch 6/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 390s 591ms/step - kl_loss: 3.0710 - reconstruction_loss: 317.5593 - total_loss: 320.6302 - val_kl_loss: 3.1502 - val_loss: 316.1778 - val_reconstruction_loss: 313.0276
Epoch 7/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 381s 589ms/step - kl_loss: 3.0640 - reconstruction_loss: 317.6821 - total_loss: 320.7462 - val_kl_loss: 3.1650 - val_loss: 315.8680 - val_reconstruction_loss: 312.7030
Epoch 8/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 375s 577ms/step - kl_loss: 3.0741 - reconstruction_loss: 317.5007 - total_loss: 320.5748 - val_kl_loss: 3.1900 - val_loss: 315.9417 - val_reconstruction_loss: 312.7516
Epoch 9/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 387s 586ms/step - kl_loss: 3.0867 - reconstruction_loss: 317.4099 - total_loss: 320.4967 - val_kl_loss: 3.1618 - val_loss: 315.8079 - val_reconstruction_loss: 312.6461
Epoch 10/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 338s 590ms/step - kl_loss: 3.0851 - reconstruction_loss: 317.4687 - total_loss: 320.5538 - val_kl_loss: 3.1679 - val_loss: 315.7830 - val_reconstruction_loss: 312.6151
Epoch 11/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 383s 590ms/step - kl_loss: 3.0895 - reconstruction_loss: 317.3818 - total_loss: 320.4711 - val_kl_loss: 3.1969 - val_loss: 315.9250 - val_reconstruction_loss: 312.7281
Epoch 12/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 372s 574ms/step - kl_loss: 3.0822 - reconstruction_loss: 317.2617 - total_loss: 320.3438 - val_kl_loss: 3.1464 - val_loss: 315.6364 - val_reconstruction_loss: 312.4900
Epoch 13/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 384s 578ms/step - kl_loss: 3.0994 - reconstruction_loss: 317.3416 - total_loss: 320.4410 - val_kl_loss: 3.1708 - val_loss: 316.0384 - val_reconstruction_loss: 312.8676
Epoch 14/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 334s 583ms/step - kl_loss: 3.0883 - reconstruction_loss: 317.0822 - total_loss: 320.1705 - val_kl_loss: 3.1625 - val_loss: 316.0277 - val_reconstruction_loss: 312.8653
Epoch 15/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4999 - reconstruction_loss: 317.4044 - kl_loss: 3.0958 - val_loss: 315.6450 - val_reconstruction_loss: 312.5350 - val_kl_loss: 3.1100
Epoch 16/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.5143 - reconstruction_loss: 317.4169 - kl_loss: 3.0972 - val_loss: 316.1226 - val_reconstruction_loss: 312.8951 - val_kl_loss: 3.2275
Epoch 17/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.5155 - reconstruction_loss: 317.4092 - kl_loss: 3.1063 - val_loss: 316.1297 - val_reconstruction_loss: 313.0135 - val_kl_loss: 3.1162
Epoch 18/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.5031 - reconstruction_loss: 317.4125 - kl_loss: 3.0903 - val_loss: 315.9903 - val_reconstruction_loss: 312.7095 - val_kl_loss: 3.2807
Epoch 19/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.5116 - reconstruction_loss: 317.4046 - kl_loss: 3.1071 - val_loss: 316.0725 - val_reconstruction_loss: 312.8803 - val_kl_loss: 3.1922
Epoch 20/50
573/573 [==============================] - 32s 56ms/step - total_loss: 320.4972 - reconstruction_loss: 317.3931 - kl_loss: 3.1040 - val_loss: 315.5361 - val_reconstruction_loss: 312.3929 - val_kl_loss: 3.1431

50 epochs:
Training VAE with latent dimension: 2
Epoch 1/50
573/573 [==============================] - 33s 55ms/step - total_loss: 320.5675 - reconstruction_loss: 317.5217 - kl_loss: 3.0459 - val_loss: 315.9824 - val_reconstruction_loss: 312.8059 - val_kl_loss: 3.1765
Epoch 2/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.5812 - reconstruction_loss: 317.5111 - kl_loss: 3.0701 - val_loss: 316.0637 - val_reconstruction_loss: 312.9071 - val_kl_loss: 3.1566
Epoch 3/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.5648 - reconstruction_loss: 317.5004 - kl_loss: 3.0646 - val_loss: 315.7896 - val_reconstruction_loss: 312.5826 - val_kl_loss: 3.2070
Epoch 4/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.5656 - reconstruction_loss: 317.4846 - kl_loss: 3.0810 - val_loss: 315.8743 - val_reconstruction_loss: 312.7462 - val_kl_loss: 3.1281
Epoch 5/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.5479 - reconstruction_loss: 317.4758 - kl_loss: 3.0723 - val_loss: 315.5905 - val_reconstruction_loss: 312.5602 - val_kl_loss: 3.0302
Epoch 6/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.5538 - reconstruction_loss: 317.4828 - kl_loss: 3.0706 - val_loss: 316.1117 - val_reconstruction_loss: 312.9886 - val_kl_loss: 3.1231
Epoch 7/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.5590 - reconstruction_loss: 317.4823 - kl_loss: 3.0765 - val_loss: 315.8457 - val_reconstruction_loss: 312.7128 - val_kl_loss: 3.1329
Epoch 8/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.5504 - reconstruction_loss: 317.4671 - kl_loss: 3.0833 - val_loss: 315.9489 - val_reconstruction_loss: 312.7222 - val_kl_loss: 3.2267
Epoch 9/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.5302 - reconstruction_loss: 317.4500 - kl_loss: 3.0803 - val_loss: 315.6910 - val_reconstruction_loss: 312.5494 - val_kl_loss: 3.1416
Epoch 10/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.5277 - reconstruction_loss: 317.4458 - kl_loss: 3.0816 - val_loss: 315.6456 - val_reconstruction_loss: 312.5508 - val_kl_loss: 3.0948
Epoch 11/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.5207 - reconstruction_loss: 317.4312 - kl_loss: 3.0899 - val_loss: 315.9359 - val_reconstruction_loss: 312.7701 - val_kl_loss: 3.1658
Epoch 12/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.5365 - reconstruction_loss: 317.4500 - kl_loss: 3.0866 - val_loss: 315.7196 - val_reconstruction_loss: 312.5393 - val_kl_loss: 3.1803
Epoch 13/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.5340 - reconstruction_loss: 317.4336 - kl_loss: 3.1004 - val_loss: 315.8693 - val_reconstruction_loss: 312.6971 - val_kl_loss: 3.1722
Epoch 14/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.5229 - reconstruction_loss: 317.4268 - kl_loss: 3.0961 - val_loss: 316.0451 - val_reconstruction_loss: 312.7950 - val_kl_loss: 3.2501
Epoch 15/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4999 - reconstruction_loss: 317.4044 - kl_loss: 3.0958 - val_loss: 315.6450 - val_reconstruction_loss: 312.5350 - val_kl_loss: 3.1100
Epoch 16/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.5143 - reconstruction_loss: 317.4169 - kl_loss: 3.0972 - val_loss: 316.1226 - val_reconstruction_loss: 312.8951 - val_kl_loss: 3.2275
Epoch 17/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.5155 - reconstruction_loss: 317.4092 - kl_loss: 3.1063 - val_loss: 316.1297 - val_reconstruction_loss: 313.0135 - val_kl_loss: 3.1162
Epoch 18/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.5031 - reconstruction_loss: 317.4125 - kl_loss: 3.0903 - val_loss: 315.9903 - val_reconstruction_loss: 312.7095 - val_kl_loss: 3.2807
Epoch 19/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.5116 - reconstruction_loss: 317.4046 - kl_loss: 3.1071 - val_loss: 316.0725 - val_reconstruction_loss: 312.8803 - val_kl_loss: 3.1922
Epoch 20/50
573/573 [==============================] - 32s 56ms/step - total_loss: 320.4972 - reconstruction_loss: 317.3931 - kl_loss: 3.1040 - val_loss: 315.5361 - val_reconstruction_loss: 312.3929 - val_kl_loss: 3.1431
Epoch 21/50
573/573 [==============================] - 32s 56ms/step - total_loss: 320.4963 - reconstruction_loss: 317.3838 - kl_loss: 3.1124 - val_loss: 315.9044 - val_reconstruction_loss: 312.7585 - val_kl_loss: 3.1459
Epoch 22/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.4879 - reconstruction_loss: 317.3768 - kl_loss: 3.1112 - val_loss: 316.0372 - val_reconstruction_loss: 312.8286 - val_kl_loss: 3.2086
Epoch 23/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.5009 - reconstruction_loss: 317.3886 - kl_loss: 3.1123 - val_loss: 315.7819 - val_reconstruction_loss: 312.5272 - val_kl_loss: 3.2547
Epoch 24/50
573/573 [==============================] - 33s 57ms/step - total_loss: 320.4819 - reconstruction_loss: 317.3642 - kl_loss: 3.1177 - val_loss: 316.2140 - val_reconstruction_loss: 312.9810 - val_kl_loss: 3.2329
Epoch 25/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4908 - reconstruction_loss: 317.3757 - kl_loss: 3.1153 - val_loss: 315.7614 - val_reconstruction_loss: 312.6194 - val_kl_loss: 3.1419
Epoch 26/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4917 - reconstruction_loss: 317.3701 - kl_loss: 3.1217 - val_loss: 316.2124 - val_reconstruction_loss: 313.1484 - val_kl_loss: 3.0640
Epoch 27/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.4939 - reconstruction_loss: 317.3639 - kl_loss: 3.1301 - val_loss: 315.7939 - val_reconstruction_loss: 312.5273 - val_kl_loss: 3.2666
Epoch 28/50
573/573 [==============================] - 32s 56ms/step - total_loss: 320.4784 - reconstruction_loss: 317.3532 - kl_loss: 3.1252 - val_loss: 315.9931 - val_reconstruction_loss: 312.7306 - val_kl_loss: 3.2625
Epoch 29/50
573/573 [==============================] - 32s 56ms/step - total_loss: 320.4737 - reconstruction_loss: 317.3497 - kl_loss: 3.1239 - val_loss: 315.8239 - val_reconstruction_loss: 312.6473 - val_kl_loss: 3.1766
Epoch 30/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4940 - reconstruction_loss: 317.3679 - kl_loss: 3.1261 - val_loss: 315.7084 - val_reconstruction_loss: 312.5164 - val_kl_loss: 3.1920
Epoch 31/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.4694 - reconstruction_loss: 317.3512 - kl_loss: 3.1181 - val_loss: 315.6398 - val_reconstruction_loss: 312.4629 - val_kl_loss: 3.1769
Epoch 32/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.4797 - reconstruction_loss: 317.3568 - kl_loss: 3.1228 - val_loss: 316.0246 - val_reconstruction_loss: 312.7561 - val_kl_loss: 3.2686
Epoch 33/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4473 - reconstruction_loss: 317.3202 - kl_loss: 3.1270 - val_loss: 315.8996 - val_reconstruction_loss: 312.6552 - val_kl_loss: 3.2445
Epoch 34/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.4658 - reconstruction_loss: 317.3318 - kl_loss: 3.1341 - val_loss: 315.8650 - val_reconstruction_loss: 312.6114 - val_kl_loss: 3.2536
Epoch 35/50
573/573 [==============================] - 32s 56ms/step - total_loss: 320.4680 - reconstruction_loss: 317.3409 - kl_loss: 3.1271 - val_loss: 316.0021 - val_reconstruction_loss: 312.8107 - val_kl_loss: 3.1915
Epoch 36/50
573/573 [==============================] - 32s 56ms/step - total_loss: 320.4592 - reconstruction_loss: 317.3230 - kl_loss: 3.1363 - val_loss: 315.9582 - val_reconstruction_loss: 312.6416 - val_kl_loss: 3.3167
Epoch 37/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.4595 - reconstruction_loss: 317.3293 - kl_loss: 3.1302 - val_loss: 315.9445 - val_reconstruction_loss: 312.7665 - val_kl_loss: 3.1780
Epoch 38/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.4565 - reconstruction_loss: 317.3186 - kl_loss: 3.1377 - val_loss: 315.5153 - val_reconstruction_loss: 312.2678 - val_kl_loss: 3.2475
Epoch 39/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.4453 - reconstruction_loss: 317.3074 - kl_loss: 3.1381 - val_loss: 315.7903 - val_reconstruction_loss: 312.7733 - val_kl_loss: 3.0170
Epoch 40/50
573/573 [==============================] - 32s 55ms/step - total_loss: 320.4567 - reconstruction_loss: 317.3182 - kl_loss: 3.1389 - val_loss: 315.6035 - val_reconstruction_loss: 312.4446 - val_kl_loss: 3.1589
Epoch 41/50
573/573 [==============================] - 32s 56ms/step - total_loss: 320.4341 - reconstruction_loss: 317.2931 - kl_loss: 3.1410 - val_loss: 315.9873 - val_reconstruction_loss: 312.7512 - val_kl_loss: 3.2362
Epoch 42/50
573/573 [==============================] - 31s 55ms/step - total_loss: 320.4394 - reconstruction_loss: 317.2946 - kl_loss: 3.1450 - val_loss: 315.5771 - val_reconstruction_loss: 312.3378 - val_kl_loss: 3.2393
Epoch 43/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4518 - reconstruction_loss: 317.3083 - kl_loss: 3.1433 - val_loss: 315.7719 - val_reconstruction_loss: 312.6403 - val_kl_loss: 3.1316
Epoch 44/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.4437 - reconstruction_loss: 317.2990 - kl_loss: 3.1445 - val_loss: 316.5700 - val_reconstruction_loss: 313.3669 - val_kl_loss: 3.2031
Epoch 45/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.4401 - reconstruction_loss: 317.2958 - kl_loss: 3.1443 - val_loss: 315.9952 - val_reconstruction_loss: 312.7653 - val_kl_loss: 3.2299
Epoch 46/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.4461 - reconstruction_loss: 317.3074 - kl_loss: 3.1385 - val_loss: 316.1116 - val_reconstruction_loss: 312.9294 - val_kl_loss: 3.1822
Epoch 47/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.4443 - reconstruction_loss: 317.2907 - kl_loss: 3.1535 - val_loss: 315.9220 - val_reconstruction_loss: 312.6057 - val_kl_loss: 3.3162
Epoch 48/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.4297 - reconstruction_loss: 317.2759 - kl_loss: 3.1537 - val_loss: 316.1551 - val_reconstruction_loss: 312.8700 - val_kl_loss: 3.2851
Epoch 49/50
573/573 [==============================] - 31s 53ms/step - total_loss: 320.4338 - reconstruction_loss: 317.2864 - kl_loss: 3.1475 - val_loss: 315.5288 - val_reconstruction_loss: 312.3201 - val_kl_loss: 3.2087
Epoch 50/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.4353 - reconstruction_loss: 317.2821 - kl_loss: 3.1530 - val_loss: 316.2529 - val_reconstruction_loss: 312.9361 - val_kl_loss: 3.3168
Training VAE with latent dimension: 5
Epoch 1/50
573/573 [==============================] - 34s 56ms/step - total_loss: 320.4142 - reconstruction_loss: 317.2626 - kl_loss: 3.1518 - val_loss: 316.0181 - val_reconstruction_loss: 312.7606 - val_kl_loss: 3.2575
Epoch 2/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4100 - reconstruction_loss: 317.2660 - kl_loss: 3.1437 - val_loss: 316.2688 - val_reconstruction_loss: 313.0245 - val_kl_loss: 3.2443
Epoch 3/50
573/573 [==============================] - 31s 53ms/step - total_loss: 320.4308 - reconstruction_loss: 317.2834 - kl_loss: 3.1473 - val_loss: 315.8512 - val_reconstruction_loss: 312.7293 - val_kl_loss: 3.1220
Epoch 4/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4244 - reconstruction_loss: 317.2669 - kl_loss: 3.1578 - val_loss: 316.0070 - val_reconstruction_loss: 312.8498 - val_kl_loss: 3.1572
Epoch 5/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4117 - reconstruction_loss: 317.2553 - kl_loss: 3.1563 - val_loss: 316.1513 - val_reconstruction_loss: 312.9677 - val_kl_loss: 3.1836
Epoch 6/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.4145 - reconstruction_loss: 317.2569 - kl_loss: 3.1577 - val_loss: 315.8134 - val_reconstruction_loss: 312.5918 - val_kl_loss: 3.2215
Epoch 7/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.4326 - reconstruction_loss: 317.2683 - kl_loss: 3.1644 - val_loss: 315.9325 - val_reconstruction_loss: 312.7073 - val_kl_loss: 3.2252
Epoch 8/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.4204 - reconstruction_loss: 317.2686 - kl_loss: 3.1518 - val_loss: 316.3854 - val_reconstruction_loss: 313.1478 - val_kl_loss: 3.2376
Epoch 9/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.4140 - reconstruction_loss: 317.2540 - kl_loss: 3.1601 - val_loss: 316.1439 - val_reconstruction_loss: 312.9881 - val_kl_loss: 3.1557
Epoch 10/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.4260 - reconstruction_loss: 317.2680 - kl_loss: 3.1581 - val_loss: 315.8463 - val_reconstruction_loss: 312.6366 - val_kl_loss: 3.2097
Epoch 11/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.4025 - reconstruction_loss: 317.2403 - kl_loss: 3.1622 - val_loss: 315.9389 - val_reconstruction_loss: 312.6253 - val_kl_loss: 3.3136
Epoch 12/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.4046 - reconstruction_loss: 317.2386 - kl_loss: 3.1657 - val_loss: 315.7869 - val_reconstruction_loss: 312.5499 - val_kl_loss: 3.2370
Epoch 13/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.4084 - reconstruction_loss: 317.2388 - kl_loss: 3.1696 - val_loss: 315.9523 - val_reconstruction_loss: 312.6789 - val_kl_loss: 3.2734
Epoch 14/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.3977 - reconstruction_loss: 317.2363 - kl_loss: 3.1613 - val_loss: 315.6578 - val_reconstruction_loss: 312.3882 - val_kl_loss: 3.2697
Epoch 15/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.4059 - reconstruction_loss: 317.2313 - kl_loss: 3.1746 - val_loss: 315.6237 - val_reconstruction_loss: 312.3541 - val_kl_loss: 3.2696
Epoch 16/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.3993 - reconstruction_loss: 317.2196 - kl_loss: 3.1796 - val_loss: 316.0205 - val_reconstruction_loss: 312.7929 - val_kl_loss: 3.2276
Epoch 17/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.4168 - reconstruction_loss: 317.2390 - kl_loss: 3.1776 - val_loss: 315.7716 - val_reconstruction_loss: 312.5063 - val_kl_loss: 3.2653
Epoch 18/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.3939 - reconstruction_loss: 317.2280 - kl_loss: 3.1662 - val_loss: 315.8194 - val_reconstruction_loss: 312.5558 - val_kl_loss: 3.2636
Epoch 19/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.4051 - reconstruction_loss: 317.2297 - kl_loss: 3.1756 - val_loss: 315.9411 - val_reconstruction_loss: 312.7844 - val_kl_loss: 3.1567
Epoch 20/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3915 - reconstruction_loss: 317.2285 - kl_loss: 3.1633 - val_loss: 315.8668 - val_reconstruction_loss: 312.6203 - val_kl_loss: 3.2465
Epoch 21/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3901 - reconstruction_loss: 317.2134 - kl_loss: 3.1767 - val_loss: 315.6084 - val_reconstruction_loss: 312.3815 - val_kl_loss: 3.2269
Epoch 22/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.4029 - reconstruction_loss: 317.2230 - kl_loss: 3.1797 - val_loss: 316.3678 - val_reconstruction_loss: 313.1366 - val_kl_loss: 3.2312
Epoch 23/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3713 - reconstruction_loss: 317.2010 - kl_loss: 3.1703 - val_loss: 315.6320 - val_reconstruction_loss: 312.2941 - val_kl_loss: 3.3379
Epoch 24/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3953 - reconstruction_loss: 317.2160 - kl_loss: 3.1793 - val_loss: 316.0714 - val_reconstruction_loss: 312.7388 - val_kl_loss: 3.3326
Epoch 25/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3927 - reconstruction_loss: 317.2156 - kl_loss: 3.1771 - val_loss: 315.8260 - val_reconstruction_loss: 312.5818 - val_kl_loss: 3.2442
Epoch 26/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3976 - reconstruction_loss: 317.2118 - kl_loss: 3.1856 - val_loss: 315.7472 - val_reconstruction_loss: 312.4206 - val_kl_loss: 3.3266
Epoch 27/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3896 - reconstruction_loss: 317.2083 - kl_loss: 3.1814 - val_loss: 316.1145 - val_reconstruction_loss: 312.8835 - val_kl_loss: 3.2309
Epoch 28/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3793 - reconstruction_loss: 317.2036 - kl_loss: 3.1758 - val_loss: 316.1119 - val_reconstruction_loss: 312.9258 - val_kl_loss: 3.1862
Epoch 29/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3925 - reconstruction_loss: 317.2132 - kl_loss: 3.1792 - val_loss: 315.7641 - val_reconstruction_loss: 312.4595 - val_kl_loss: 3.3046
Epoch 30/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3786 - reconstruction_loss: 317.1958 - kl_loss: 3.1826 - val_loss: 315.9868 - val_reconstruction_loss: 312.8195 - val_kl_loss: 3.1673
Epoch 31/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3851 - reconstruction_loss: 317.2022 - kl_loss: 3.1830 - val_loss: 315.8433 - val_reconstruction_loss: 312.5437 - val_kl_loss: 3.2996
Epoch 32/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3925 - reconstruction_loss: 317.2145 - kl_loss: 3.1781 - val_loss: 315.9690 - val_reconstruction_loss: 312.7669 - val_kl_loss: 3.2021
Epoch 33/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3824 - reconstruction_loss: 317.1860 - kl_loss: 3.1963 - val_loss: 315.9501 - val_reconstruction_loss: 312.7474 - val_kl_loss: 3.2027
Epoch 34/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3651 - reconstruction_loss: 317.1707 - kl_loss: 3.1944 - val_loss: 315.6488 - val_reconstruction_loss: 312.3531 - val_kl_loss: 3.2957
Epoch 35/50
573/573 [==============================] - 30s 51ms/step - total_loss: 320.3781 - reconstruction_loss: 317.1838 - kl_loss: 3.1944 - val_loss: 315.8713 - val_reconstruction_loss: 312.6734 - val_kl_loss: 3.1979
Epoch 36/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.3719 - reconstruction_loss: 317.1848 - kl_loss: 3.1870 - val_loss: 315.7000 - val_reconstruction_loss: 312.4638 - val_kl_loss: 3.2362
Epoch 37/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3707 - reconstruction_loss: 317.1829 - kl_loss: 3.1877 - val_loss: 316.1816 - val_reconstruction_loss: 312.9386 - val_kl_loss: 3.2429
Epoch 38/50
573/573 [==============================] - 29s 50ms/step - total_loss: 320.3586 - reconstruction_loss: 317.1725 - kl_loss: 3.1861 - val_loss: 315.9960 - val_reconstruction_loss: 312.8348 - val_kl_loss: 3.1611
Epoch 39/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3706 - reconstruction_loss: 317.1843 - kl_loss: 3.1864 - val_loss: 315.8897 - val_reconstruction_loss: 312.6135 - val_kl_loss: 3.2762
Epoch 40/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3671 - reconstruction_loss: 317.1837 - kl_loss: 3.1832 - val_loss: 315.6408 - val_reconstruction_loss: 312.3698 - val_kl_loss: 3.2710
Epoch 41/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3731 - reconstruction_loss: 317.1833 - kl_loss: 3.1899 - val_loss: 316.1378 - val_reconstruction_loss: 312.9771 - val_kl_loss: 3.1607
Epoch 42/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3661 - reconstruction_loss: 317.1675 - kl_loss: 3.1988 - val_loss: 316.0446 - val_reconstruction_loss: 312.8412 - val_kl_loss: 3.2034
Epoch 43/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.3563 - reconstruction_loss: 317.1619 - kl_loss: 3.1945 - val_loss: 316.3860 - val_reconstruction_loss: 313.1736 - val_kl_loss: 3.2125
Epoch 44/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3590 - reconstruction_loss: 317.1631 - kl_loss: 3.1955 - val_loss: 315.6469 - val_reconstruction_loss: 312.3979 - val_kl_loss: 3.2491
Epoch 45/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3613 - reconstruction_loss: 317.1678 - kl_loss: 3.1931 - val_loss: 316.0490 - val_reconstruction_loss: 312.8340 - val_kl_loss: 3.2150
Epoch 46/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3429 - reconstruction_loss: 317.1437 - kl_loss: 3.1988 - val_loss: 315.8210 - val_reconstruction_loss: 312.6005 - val_kl_loss: 3.2205
Epoch 47/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3575 - reconstruction_loss: 317.1613 - kl_loss: 3.1962 - val_loss: 315.6760 - val_reconstruction_loss: 312.3850 - val_kl_loss: 3.2910
Epoch 48/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3424 - reconstruction_loss: 317.1443 - kl_loss: 3.1979 - val_loss: 315.9135 - val_reconstruction_loss: 312.7431 - val_kl_loss: 3.1703
Epoch 49/50
573/573 [==============================] - 29s 51ms/step - total_loss: 320.3588 - reconstruction_loss: 317.1541 - kl_loss: 3.2047 - val_loss: 316.1319 - val_reconstruction_loss: 312.8813 - val_kl_loss: 3.2506
Epoch 50/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3501 - reconstruction_loss: 317.1575 - kl_loss: 3.1926 - val_loss: 315.9687 - val_reconstruction_loss: 312.6462 - val_kl_loss: 3.3225
Training VAE with latent dimension: 10
Epoch 1/50
573/573 [==============================] - 34s 56ms/step - total_loss: 320.3539 - reconstruction_loss: 317.1653 - kl_loss: 3.1885 - val_loss: 315.6079 - val_reconstruction_loss: 312.3782 - val_kl_loss: 3.2297
Epoch 2/50
573/573 [==============================] - 30s 53ms/step - total_loss: 320.3462 - reconstruction_loss: 317.1392 - kl_loss: 3.2070 - val_loss: 315.8697 - val_reconstruction_loss: 312.6190 - val_kl_loss: 3.2506
Epoch 3/50
573/573 [==============================] - 30s 52ms/step - total_loss: 320.3620 - reconstruction_loss: 317.1544 - kl_loss: 3.2075 - val_loss: 316.3815 - val_reconstruction_loss: 313.2204 - val_kl_loss: 3.1611
Epoch 4/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.3370 - reconstruction_loss: 317.1332 - kl_loss: 3.2038 - val_loss: 315.4990 - val_reconstruction_loss: 312.1968 - val_kl_loss: 3.3022
Epoch 5/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.3466 - reconstruction_loss: 317.1453 - kl_loss: 3.2013 - val_loss: 316.4372 - val_reconstruction_loss: 313.1114 - val_kl_loss: 3.3259
Epoch 6/50
573/573 [==============================] - 31s 53ms/step - total_loss: 320.3578 - reconstruction_loss: 317.1441 - kl_loss: 3.2139 - val_loss: 315.9591 - val_reconstruction_loss: 312.7265 - val_kl_loss: 3.2326
Epoch 7/50
573/573 [==============================] - 31s 54ms/step - total_loss: 320.3488 - reconstruction_loss: 317.1480 - kl_loss: 3.2012 - val_loss: 315.9326 - val_reconstruction_loss: 312.6975 - val_kl_loss: 3.2352
Epoch 8/50
572/573 [============================>.] - ETA: 0s - total_loss: 320.3362 - reconstruction_loss: 317.1322 - kl_loss: 3.2039

Discusion:
	The difference between the multiple latent spaces are mainly in the speeds and amount of loss. The smaller the latent space is the faster it was to train. However the bigger the latent space, the more accurate the train was based on the fact that the total loss started at a lower number and continued to drop. The amount of epochs also determined the speed and accuracy of the training. The first two tests I ran (5 epochs and 20 epochs) I used the GPU and for the 50 epochs  I used the TPU so I compared the times separately across the two different groups. The observations continued to be the same even though my computer crashed during the 50 epochs test. However I reported the numbers I was able to obtain from the test. The lower epochs once again ran faster overall than the larger ones. The larger epochs, however, ended up having a smaller amount of total loss over all even if they started out with the same or higher amount of loss initially.
